In [1]:
%matplotlib inline

# import packages
import numpy as np
import time

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib import gridspec

from astropy.timeseries import LombScargle
from scipy.optimize import minimize

import celerite
import corner
import emcee


In [2]:
# define plot properties
from cycler import cycler
import matplotlib.cm as cm

from matplotlib import rcParams
from matplotlib import rc
from mpl_toolkits.axes_grid1 import make_axes_locatable

def rgb(r,g,b):
    return (float(r)/256.,float(g)/256.,float(b)/256.)

cb2 = [rgb(31,120,180), rgb(255,127,0), rgb(51,160,44), rgb(227,26,28), \
       rgb(166,206,227), rgb(253,191,111), rgb(178,223,138), rgb(251,154,153)]

rcParams['figure.figsize'] = (9,7.5)

rcParams['lines.linewidth'] = 1

rcParams['axes.prop_cycle'] = cycler('color', cb2)
rcParams['axes.facecolor'] = 'white'
rcParams['axes.grid'] = False

rcParams['patch.facecolor'] = cb2[0]
rcParams['patch.edgecolor'] = 'white'

rcParams['font.size'] = 23


## A single AGN.

> Define useful function for CAR(1) model.

In [ ]:
import numpy as np

#--------------------------------------------------------------------------------
# draw CAR1 parameters
def draw_car1_params():
    """
    Based on the parameter distributions found by Kelly et al. (2009), Table 2 Fig 5
    """
    tau = 10**np.random.normal(2.75, 0.66)
    sig = 10**np.random.normal(-2.04, 0.23)
    return tau, sig

# generate CAR1 covariance matrix
def generate_car1_cov(t, tau, sigma):
    r = np.fabs(t[:,None] - t[None,:])
    var = 0.5*tau*sigma**2
    cov = var*np.exp(-r/tau)
    return cov

#--------------------------------------------------------------------------------
# generate time vector
def generate_time_vector(cadence, season_length, n_seasons):
    """
    Generates an observation time vector for n_seasons, each of
    season_length days. The observations within each season will
    be spaced by cadence +/- 1 day.

    Also returns a reference vector sampled daily over the same
    duration to help when plotting true light curves
    """
        
    duration = n_seasons*season_length
    n_obs = int(duration/cadence)

    dt = np.random.uniform(cadence-1.0,cadence+1.0,n_obs)
    t = np.cumsum(dt)

    seasons = []
    on = True
    
    for s in range(n_seasons):
        if on is True:
            seasons.append(t[(t>s*season_length)*(t<(s+1)*season_length)])
            on = False
        else:
            on = True
    t_obs = np.hstack(seasons)
    t_clean = np.linspace(0, duration, int(duration))
    
    return t_obs, t_clean

#--------------------------------------------------------------------------------
# generate light curve
def generate_noiseless_light_curve(t_obs, t_clean, c, tau, sigma):

    n_obs = len(t_obs)
    n_clean = len(t_clean)
    
    t = np.hstack([t_obs, t_clean])
    n = len(t)

    c = c*np.ones((n,)) # mean magnitude
    K = generate_car1_cov(t, tau, sigma)

    y = np.random.multivariate_normal(c,K)
    y_obs = y[0:n_obs]
    y_clean = y[n_obs:]

    return y_obs, y_clean

#--------------------------------------------------------------------------------
# add gaussian noise
def add_gaussian_noise(y_obs, std):
    return y_obs + np.random.normal(0.0, std, len(y_obs))

#--------------------------------------------------------------------------------
# log likelihood as MLE objective function
def neg_log_like(params, y, gp):
    gp.set_parameter_vector(params)
    return -gp.log_likelihood(y)

# log likelihood for dynesty
def log_like(params, y, gp):
    gp.set_parameter_vector(params)
    return gp.log_likelihood(y)

# create random uniform variable within bound where u is from from U[0,1]
def prior_tform(u, gp):
    bounds = gp.get_parameter_bounds()
    return [b[0] + u[i]*(b[1] - b[0]) for i,b in enumerate(bounds)]

# log posterior for emcee
def log_post(params, y, gp):
    gp.set_parameter_vector(params)
    lp = gp.log_prior()
    if not np.isfinite(lp):
        return -np.inf
    return gp.log_likelihood(y)

> Make CAR(1) models.

In [ ]:
# observation properties
cad = 3.0 #Cadence in days (time steps will be this +/- 1 day)
seas_len = 180 #Season length
n_seas = 3 #Number of seasons
err = 0.1 #Magnitude of Gaussian errors to add to observations

#Set time series properties
c = np.random.uniform(16,18)
tau, sig = draw_car1_params()
print(c, tau, sig)

#Generate light curves
tHat, tTrue = generate_time_vector(cad, seas_len, n_seas)
yHat, yTrue = generate_noiseless_light_curve(tHat, tTrue, c, tau, sig)
#yHat = add_gaussian_noise(yHat, err)
print(tTrue)

# plot the light curve
plt.plot(tHat,yHat)
plt.plot(tTrue,yTrue)

> Calculate Lomb Scargle.

In [ ]:
#Compute Lomb-Scargle and true power spectra (LS won't be well normalised
fLS, pLS = LombScargle(tTrue, yTrue, normalization="psd").autopower(maximum_frequency=0.5)
fHatLS, pHatLS = LombScargle(tHat, yHat, dy=err, normalization="psd").autopower(maximum_frequency=0.2)
fTrue = np.logspace(np.log10(min(fLS)), np.log10(max(fLS)))
pTrue = 2*sig**2*tau**2/(1 + (2*np.pi*tau*fTrue)**2)

# plot power spectrum
#Plot power spectra
plt.plot(fTrue,pTrue, label="truth")
plt.plot(fLS,pLS,"k", label="Lomb-Scargle (truth)")
plt.plot(fHatLS,pHatLS,"r", label="Lomb-Scargle (data)")

plt.xscale("log")
plt.yscale("log")
plt.ylabel("power")
plt.xlabel("frequency")
plt.legend(loc="best")


> First initialize the optimization.

In [ ]:
#MLE hyperparameter fit
bounds = {"log_a": (np.log(1e-6),np.log(1e3)), "log_c": (np.log(1e-4),np.log(1e-1))}
kernel = celerite.terms.RealTerm(log_a=np.log(0.5*tau*sig**2), log_c=np.log(1/tau), bounds=bounds)
gp = celerite.GP(kernel, np.mean(yTrue))
gp.compute(tTrue, yerr=err)
init = gp.get_parameter_vector()
bounds = gp.get_parameter_bounds()
r = minimize(neg_log_like, init, method="L-BFGS-B", bounds=bounds, args=(yTrue, gp))
print(r)

> Evaluate with MCMC.

In [ ]:
#MCMC using emcee
init = r.x
ndim, nwalkers = len(init), 32
init = r.x + np.random.normal(0, 1e-5, (nwalkers, ndim))
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_post, args=(yTrue, gp))
sampler.run_mcmc(init, 2000)
samples = sampler.flatchain

# make the corner plot
corner.corner(samples, truths=[np.log(0.5*tau*sig**2),np.log(1.0/tau)], color="orange",\
              labels=["$\log(\\tau\sigma^2/2)$","$\log(\\tau)$"], show_titles=True)


> Compute the posterior distribution of power spectrum and model prediciton.

In [ ]:
# prediction of the light curve
gp.set_parameter_vector(np.median(samples, axis=0))
yPred, varPred = gp.predict(yHat, tTrue, return_var=True)
stdPred = np.sqrt(varPred)

# prediction of the power spectrum
pMCMC = []
for s in samples:
    gp.set_parameter_vector(s)
    #There is a weird factor in this to get the normalisation right
    pMCMC.append(2*np.sqrt(2*np.pi)*kernel.get_psd(2*np.pi*fTrue))
    
pMCMC = np.vstack(pMCMC)
pMCMCmed = np.median(pMCMC,axis=0)
pMCMCperc = np.percentile(pMCMC,[2.5,16,84,97.5],axis=0)

> Plot results for the light curves.

In [ ]:
#Plot light curves
plt.plot(tTrue, yTrue, label="truth")
plt.errorbar(tHat, yHat, yerr=err, color="r", marker="x", linestyle="", label="data")
plt.plot(tTrue, yPred, color="orange", label="GP reconstruction")
plt.fill_between(tTrue, yPred+2*stdPred, yPred-2*stdPred, color="orange", alpha=0.3)
plt.fill_between(tTrue, yPred+stdPred, yPred-stdPred, color="orange", alpha=0.5)
plt.gca().invert_yaxis()
plt.legend(loc="best")
plt.ylabel("mag")
plt.xlabel("time")


In [ ]:
#Plot power spectra
plt.plot(fTrue,pTrue, label="truth")
plt.plot(fLS,pLS,"k", label="Lomb-Scargle (truth)")
plt.plot(fHatLS,pHatLS,"r", label="Lomb-Scargle (data)")
plt.plot(fTrue, pMCMCmed,color="orange", label="GP reconstruction")
plt.fill_between(fTrue, pMCMCperc[0], pMCMCperc[3], color="orange", alpha=0.3)
plt.fill_between(fTrue, pMCMCperc[1], pMCMCperc[2], color="orange", alpha=0.5)
plt.xscale("log")
plt.yscale("log")
plt.ylabel("power")
plt.xlabel("frequency")
plt.legend(loc="best")

## Dual AGNs.

> Make models.

In [ ]:
def generate_noiseless_double_light_curve(t_obs, t_clean, c1, c2, tau1, tau2, sigma1, sigma2):

    n_obs = len(t_obs)
    n_clean = len(t_clean)
    
    t = np.hstack([t_obs, t_clean])
    n = len(t)

    c1 = c1*np.ones((n,))
    c2 = c2*np.ones((n,))
    K1 = generate_car1_cov(t, tau1, sigma1)
    K2 = generate_car1_cov(t, tau2, sigma2)

    m1 = np.random.multivariate_normal(c1,K1)
    m2 = np.random.multivariate_normal(c2,K2)
    f1 = 10**((18-m1)/2.5)
    f2 = 10**((18-m2)/2.5)
    f = f1 + f2
    y = 18-2.5*np.log10(f)
    y_obs = y[0:n_obs]
    y_clean = y[n_obs:]

    return y_obs, y_clean, m1[n_obs:], m2[n_obs:]

> Fit with a single CAR(1), and show results as for the 1 AGN case.

In [ ]:
#Set up "observing properties"
cad = 3.0 #Cadence in days (time steps will be this +/- 1 day)
seas_len = 180 #Season length
n_seas = 3 #Number of seasons
err = 0.1 #Magnitude of Gaussian errors to add to observations

#-------------------------------------------------------------------------------------------
#Set properties of two component time series
c1, c2 = np.random.uniform(16,18,2)
tau1, sig1 = draw_car1_params()
tau2, sig2 = draw_car1_params()
print(c1, tau1, sig1)
print(c2, tau2, sig2)

#-------------------------------------------------------------------------------------------
#Parameter truths (useful for plotting)
truths1 = [np.log(0.5*tau1*sig1**2),np.log(1.0/tau1),np.log(0.5*tau1*sig1**2),np.log(1.0/tau1)]
truths2 = [np.log(0.5*tau2*sig2**2),np.log(1.0/tau2),np.log(0.5*tau2*sig2**2),np.log(1.0/tau2)]

#-------------------------------------------------------------------------------------------
#Generate light curves
tHat, tTrue = generate_time_vector(cad, seas_len, n_seas)
yHat, yTrue, m1, m2 = generate_noiseless_double_light_curve(tHat, tTrue, c1, c2, tau1, tau2, sig1, sig2)
yHat = add_gaussian_noise(yHat, err)

#Calculate periodogram of clean light curve
fLS, pLS = LombScargle(tTrue, yTrue, normalization="psd").autopower(maximum_frequency=0.5)
fHatLS, pHatLS = LombScargle(tHat, yHat, dy=err, normalization="psd").autopower(maximum_frequency=0.2)
fTrue = np.logspace(np.log10(min(fLS)), np.log10(max(fLS)))

#-------------------------------------------------------------------------------------------
#MLE (fitting single CAR(1) to mixed light curve)
bounds = {"log_a": (np.log(1e-6),np.log(1e3)), "log_c": (np.log(1e-4),np.log(1e-1))}
kernel = celerite.terms.RealTerm(log_a=np.log(0.5*tau1*sig1**2), log_c=np.log(1/tau1), bounds=bounds)
gp = celerite.GP(kernel, np.mean(yHat))
gp.compute(tHat, yerr=err)
init = gp.get_parameter_vector()
bounds = gp.get_parameter_bounds()
r = minimize(neg_log_like, init, method="L-BFGS-B", bounds=bounds, args=(yHat, gp))
print(r)

#MCMC
init = r.x
ndim, nwalkers = len(init), 32
init = r.x + np.random.normal(0, 1e-5, (nwalkers, ndim))
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_post, args=(yHat, gp))
sampler.run_mcmc(init, 2000)
samples = sampler.flatchain

#Corner plot
fig = corner.corner(samples, color="orange", labels=["$\log(\\tau\sigma^2/2)$","$\log(\\tau)$"], show_titles=True)

#-------------------------------------------------------------------------------------------
ax = np.array(fig.axes).reshape((ndim, ndim))
for i in range(ndim):
	axis = ax[i,i]
	axis.axvline(truths1[i], color="g")
	axis.axvline(truths2[i], color="purple")
	for j in range(i):
		axis = ax[i,j]
		axis.axvline(truths1[j], color="g")
		axis.axvline(truths2[j], color="purple")
		axis.axhline(truths1[i], color="g")
		axis.axhline(truths2[i], color="purple")
		axis.plot(truths1[j], truths1[i], color="g", marker="s")
		axis.plot(truths2[j], truths2[i], color="purple", marker="s")

In [ ]:
#Compute posterior power spectrum
gp.set_parameter_vector(np.median(samples, axis=0))
yPred, varPred = gp.predict(yHat, tTrue, return_var=True)
stdPred = np.sqrt(varPred)
pMCMC = []
for s in samples:
	gp.set_parameter_vector(s)
	pMCMC.append(2*np.sqrt(2*np.pi)*kernel.get_psd(2*np.pi*fTrue))
pMCMC = np.vstack(pMCMC)
pMCMCmed = np.median(pMCMC,axis=0)
pMCMCperc = np.percentile(pMCMC,[2.5,16,84,97.5],axis=0)

#Plot light curves
plt.plot(tTrue, yTrue, label="truth")
plt.plot(tTrue, m1, "g", label="component 1")
plt.plot(tTrue, m2, color="purple", label="component 2")
plt.errorbar(tHat, yHat, yerr=err, color="r", marker="x", linestyle="", label="data")
plt.plot(tTrue, yPred, color="orange", label="GP reconstruction")
plt.fill_between(tTrue, yPred+2*stdPred, yPred-2*stdPred, color="orange", alpha=0.3)
plt.fill_between(tTrue, yPred+stdPred, yPred-stdPred, color="orange", alpha=0.5)
plt.gca().invert_yaxis()
plt.xlabel("mjd")
plt.ylabel("mag")
plt.legend(loc="best")
plt.show()


In [ ]:
#Plot power spectra
p1 = 2*sig1**2*tau1**2/(1 + (2*np.pi*tau1*fTrue)**2)
p2 = 2*sig2**2*tau2**2/(1 + (2*np.pi*tau2*fTrue)**2)
plt.plot(fTrue,p1, color="g", label="component 1")
plt.plot(fTrue,p2, color="purple", label="component 2")
plt.plot(fTrue, pMCMCmed,color="orange", label="GP reconstruction")
plt.fill_between(fTrue, pMCMCperc[0], pMCMCperc[3], color="orange", alpha=0.3)
plt.fill_between(fTrue, pMCMCperc[1], pMCMCperc[2], color="orange", alpha=0.5)
plt.xscale("log")
plt.yscale("log")
plt.xlabel("frequency")
plt.ylabel("power")
plt.legend(loc="best")
